In [1]:
from glob import glob
from tqdm import tqdm
import json
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

files = sorted(glob('/home/husein/ssd3/online-articles-partition/*.tokenized'))
len(files)

229

In [2]:
extra = ['/home/ubuntu/dedup-text-dataset/wikipedia-2023-10-01.jsonl',
 '/home/ubuntu/dedup-text-dataset/wikipedia-20230901.en.filtered.jsonl',
 '/home/ubuntu/dedup-text-dataset/dictionary.jsonl',
 '/home/ubuntu/dedup-text-dataset/dewanbahasa-jdbp.jsonl',
 '/home/ubuntu/dedup-text-dataset/dialect.jsonl',
 '/home/ubuntu/dedup-text-dataset/kamusbm.jsonl',
 '/home/ubuntu/dedup-text-dataset/wiktionary-bahasa.jsonl',
 '/home/ubuntu/dedup-text-dataset/hansard.jsonl',
 '/home/ubuntu/dedup-text-dataset/lom.agc.gov.my.jsonl',
 '/home/ubuntu/dedup-text-dataset/parlimen-gov.jsonl',
 '/home/ubuntu/dedup-text-dataset/gov.my.jsonl',
 '/home/ubuntu/dedup-text-dataset/edu.my.jsonl'
]
# for f in extra:
#     f = os.path.split(f)[1]
#     url = f'https://huggingface.co/datasets/malaysia-ai/online-articles-partition/resolve/main/{f}.tokenized'
#     os.system(f'wget {url}')

In [3]:
selected = [
    'e-penerbitan.jsonl.tokenized',
    'fintechnews.my.jsonl.tokenized',
    'maukerja.my.jsonl.tokenized',
    'ekonomirakyat.com.jsonl.tokenized',
    'asklegal.jsonl.tokenized'
]
files = [f for f in files if any([s in f for s in selected])]

for f in extra:
    f = os.path.split(f)[1]
    files.append(f'{f}.tokenized')
    
len(files)

17

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=4096,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.0,
    do_sample=True,
)

client = InferenceClient(
    '', timeout = 50
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
partitions = []
total = 0
for f in tqdm(files):
    total = 0
    filename = os.path.split(f)[1].replace('.tokenized', '')
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            partitions.append((l, filename))
            total += 1
    
len(partitions)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.75it/s]


531164

In [6]:
with open('random-reply-user.json') as fopen:
    random_user = json.load(fopen)
    
with open('stupid-random-reply-user.json') as fopen:
    stupid_random_user = json.load(fopen)
    
disagree = [
    'hi', 'hi hi', 'saya nak tanya soalan boleh', 'hello'
    'bullshit', 'u are lying', 'tipu la ko', 'penipu', 'tipu lah sial', 'ko tipu', 'aku tak caye',
    'wtf tipu', 'ye ke ko ni, tipu lah', 'tipu sial', 'bodoh salah', 'tak betul la sial',
    'tak betul', 'salah', 'ko pasti ke ni', 'tipu tipu tipu',
]

with open('disagree.json') as fopen:
    disagree.extend(json.load(fopen)[:1000])
    
with open('doubt.json') as fopen:
    disagree.extend(json.load(fopen)[:1000])
    
disagree = [d for d in disagree if len(d) > 2]
len(disagree)

2018

In [7]:
with open('malay-instructions.json') as fopen:
    malay_instructions = json.load(fopen)
    
len(malay_instructions)

605702

In [9]:
!mkdir rag-multiturn-chaotic

mkdir: cannot create directory ‘rag-multiturn-chaotic’: File exists


In [10]:
# q = partitions[0]

In [11]:
# prompt = f'{q[0]}\n\ngenerate questions based on context above'
# formatted_prompt = format_prompt(prompt, [])

# while True:
#     try:
#         stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
#         output = stream.generated_text
#         break
#     except:
#         pass

# splitted = output.strip().split('\n')
# splitted = [s for s in splitted if '.' if s and '?' in s]
# splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
# splitted = [s for s in splitted if len(s) > 3]
# splitted = [s[1:] if s[0] == '"' else s for s in splitted]
# questions = [s[:-1] if s[-1] == '"' else s for s in splitted]
# questions = [s for s in questions if len(s) > 20]

In [12]:
# history = []
# for _ in range(random.randint(2, 5)):
#     choice = random.randint(0, 5)
#     if choice in [0,1]:
#         rq = random.choice(random_user)
#     elif choice in [2]:
#         rq = random.choice(disagree)
#     elif choice in [3]:
#         rq = random.choice(stupid_random_user)
#     else:
#         rq = random.choice(questions)
#     u = f"""
# knowledge base is below.
# ---------------------
# {q[0]}
# ---------------------
# Given the knowledge base and not prior knowledge, respond user question.
# Question: {rq}
# """
#     formatted_prompt = format_prompt(u, history)
#     while True:
#         try:
#             stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
#             a = stream.generated_text.strip()
#             break
#         except Exception as e:
#             # print(e)
#             pass

#     history.append((rq, a))
    
# history

In [26]:
def answer(q, i):
    filename = f'rag-multiturn-chaotic/{i}.json'
    if os.path.exists(filename):
        return
    
    prompt = f'{q[0]}\n\ngenerate questions based on context above'
    formatted_prompt = format_prompt(prompt, [])
    
    while True:
        try:
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            break
        except:
            pass
    
    splitted = output.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    questions = [s[:-1] if s[-1] == '"' else s for s in splitted]
    questions = [s for s in questions if len(s) > 20]
    
    if not len(questions):
        with open(filename, 'w') as fopen:
            json.dump(False, fopen)
        return
    
    history = []
    for _ in range(random.randint(2, 4)):
        choice = random.randint(0, 7)
        if choice in [0]:
            rq = random.choice(random_user)
        elif choice in [1]:
            rq = random.choice(disagree)
        elif choice in [2]:
            rq = random.choice(stupid_random_user)
        elif choice in [3]:
            rq = random.choice(malay_instructions)
        else:
            rq = random.choice(questions)
            
        u = f"""
knowledge base is below.
---------------------
{q[0]}
---------------------
Given the knowledge base and not prior knowledge, respond user question.
Question: {rq}
"""
        formatted_prompt = format_prompt(u.strip(), history)
        while True:
            try:
                stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
                a = stream.generated_text.strip()
                break
            except Exception as e:
                # print(e)
                pass

        history.append((rq, a))
    
    d = {
        'filename': q[1],
        'context': q[0],
        'questions': questions,
        'multiturn': history
    }
    
    with open(filename, 'w') as fopen:
        json.dump(d, fopen)

In [27]:
answer(partitions[0], 0)

In [28]:
with open('rag-multiturn-chaotic/0.json') as fopen:
    d = json.load(fopen)

In [16]:
d['multiturn']

[['Can fish ride bicycles?',
  "Based on the information provided in the knowledge base, there is no mention of fish being able to ride bicycles. In fact, the knowledge base focuses on information related to Allianz's referral program and Budget 2022's initiatives. Therefore, I can confidently say that fish cannot ride bicycles."],
 ['Why does Malaysia still have such strict internet regulations, when other countries are embracing the digital age and promoting freedom of information?',
  'Malaysia\'s internet regulations are governed by the Communications and Multimedia Act (CMA) 1998, which includes provisions for content regulation and restrictions on "indecent, obscene, false, menacing or offensive" content. The Malaysian Communications and Multimedia Commission (MCMC) is responsible for enforcing these regulations. While other countries may have more relaxed regulations, Malaysia\'s stance is based on its unique societal norms, cultural values, and legal frameworks. Malaysia has be

In [17]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [18]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(partitions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 600
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 62%|███████████████████████████████████████████████████████████▉                                    | 331526/531164 [11:10:12<29:43:50,  1.87it/s]

In [3]:
files = glob('rag-multiturn-chaotic/*.json')

In [10]:
all_texts = set()
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    if not data:
        continue
    for d in data['multiturn']:
        all_texts |= set(d)
len(all_texts)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 331082/331082 [00:08<00:00, 40173.15it/s]


1596554

In [11]:
with open('rag-multiturn-chaotic.texts', 'w') as fopen:
    for t in all_texts:
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [13]:
!cp rag-multiturn-chaotic.texts ../translation

In [14]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/rag-multiturn-chaotic.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

1381323

In [15]:
with open('rag-multiturn-chaotic.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if not data:
            continue
            
        chat = []
        for d in data['multiturn']:
            chat.extend([
                {'role': 'user', 'content': d[0], 'content_ms': mapping.get(d[0])},
                {'role': 'assistant', 'content': d[1], 'content_ms': mapping.get(d[1])}
            ])

        d = {
            'filename': data['filename'],
            'chat': chat,
            'context': data['context'],
            'questions': data['questions']
        }
        fopen_l.write(f'{json.dumps(d)}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 331082/331082 [01:31<00:00, 3633.69it/s]


In [16]:
!head -n 1 rag-multiturn-chaotic.jsonl

{"filename": "wikipedia-20230901.en.filtered.jsonl", "chat": [{"role": "user", "content": "Why have some analysts suggested that the Malaysian government's anti-corruption efforts have been selective and politically motivated, and what implications does this have for the rule of law and democratic governance in the country?", "content_ms": "Mengapakah beberapa penganalisis mencadangkan bahawa usaha pencegahan rasuah kerajaan Malaysia adalah secara terpilih dan bermotifkan politik, dan apakah implikasinya terhadap kedaulatan undang-undang dan tadbir urus demokrasi di negara ini?"}, {"role": "assistant", "content": "I'm sorry for any inconvenience, but the provided knowledge base does not contain any information about the Malaysian government's anti-corruption efforts, or the rule of law and democratic governance in Malaysia. Therefore, I am unable to respond to this question without additional information. However, I can provide information on these topics in general if that would be he

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='rag-multiturn-chaotic.jsonl',
    path_in_repo='mrag-multiturn-chaotic.jsonl',
    repo_id='mesolitica/mixtral-malaysian-rag',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


rag-multiturn-chaotic.jsonl:   0%|          | 0.00/4.24G [00:00<?, ?B/s]